In [12]:
import numpy as np
import csv
import re
import math
import enchant
d = enchant.Dict("en_US")

In [220]:
#creating vocabulary list
#cleaning data
#storing in variables 
tweets_pos= list()
tweets_neg= list()
tweets_neu= list()
vocab_pos = list()
count_pos = list()
vocab_neg = list()
count_neg = list()
vocab_neu = list()
count_neu = list()
XTE_pos = list()
XTE_neg = list()
XTE_neu = list()
XTR_pos = list()
XTR_neg = list()
XTR_neu = list()
T_vocab = list()

m=0 #number of data examples/tweets
stop_words = ['a', 'an', 'the', 'it','i','to','in','my','am','is','if','for','be','and','im' ,'you','herself', 'it', 'its', 'been', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'of', 'at', 'by', 'for', 'with', 'to', 'from']
exclude= ['-','@','#','',':',';','...','.','..','!','/','']
tweet = 0

with open('/Users/syedhadi/Desktop/Assignment4/Tweets.csv') as file:
    iterator = csv.reader(file, delimiter=',')
    for text in iterator:
        if tweet == 0:
            tweet += 1
        else:
            st1 = text[1].lower()
            st1=st1.replace('@',' ')
            st1=st1.replace('#','')
            st1=st1.replace('$','')
            st1=st1.replace(':',' ')
            st1=st1.replace(';',' ')
            st1=st1.replace('/','')
            st1=st1.replace('.',' ')
            st1=st1.replace('&',' ')
            st1=st1.replace('-',' ')
            st1=st1.replace('=','')
            st1=st1.replace('!','')
            st1=st1.replace('?','')
            st1=st1.replace(',','')
            st1=st1.replace('"','')
            st1=st1.replace('~',' ')
            st1=st1.replace('(','')
            st1=st1.replace(')','')
            st1=st1.replace('+','')
            st1=st1.replace('*',' ')
            st1=st1.replace('^',' ')
            st1=st1.replace('“','')
            st1=st1.replace('‘','')
            st1=st1.replace("'","")
            st1=st1.replace("{","")
            st1=st1.replace("}","")
            
            if(text[0]=='positive'):
                tweets_pos.append(st1)
            if(text[0]=='negative'):
                tweets_neg.append(st1)
            if(text[0]=='neutral'):
                tweets_neu.append(st1)            
                    
            tweet += 1
            
#Train Test Split

for i in range (len(tweets_pos)):
    if i > (int(0.8*(len(tweets_pos)))):
        XTE_pos.append(tweets_pos[i])
    else:
        XTR_pos.append(tweets_pos[i])
for i in range (len(tweets_neg)):
    if i > (int(0.8*(len(tweets_neg)))):
        XTE_neg.append(tweets_neg[i])
    else:
        XTR_neg.append(tweets_neg[i])
for i in range (len(tweets_neu)):
    if i > (int(0.8*(len(tweets_neu)))):
        XTE_neu.append(tweets_neu[i])
    else:
        XTR_neu.append(tweets_neu[i])


tweets_pos=None
tweets_neg=None
tweets_neu=None

#Data cleaning and attributing

#Creating Test data

for eg in XTR_pos:
    words= eg.split()
    for i in words:
        if i in stop_words or i.isnumeric() or i in exclude:
            continue
        if d.check(i):
            if i in vocab_pos:
                count_pos[vocab_pos.index(i)]+=1
            else:
                vocab_pos.append(i)
                count_pos.append(1)
        else:
            if (d.suggest(i)):
                suggested = ((d.suggest(i)[0]).lower()).split()
                for subword in suggested:
                    if subword in vocab_pos:
                        count_pos[vocab_pos.index(subword)]+=1
                    else:
                        vocab_pos.append(subword)
                        count_pos.append(1)
        if i not in T_vocab:
            T_vocab.append(i)
                
                
for eg in XTR_neg:
    words= eg.split()
    for i in words:
        if i in stop_words or i.isnumeric() or i in exclude:
            continue
        if d.check(i):
            if i in vocab_neg:
                count_neg[vocab_neg.index(i)]+=1
            else:
                vocab_neg.append(i)
                count_neg.append(1)
        else:
            if (d.suggest(i)):
                suggested = ((d.suggest(i)[0]).lower()).split()
                for subword in suggested:
                    if subword in vocab_neg:
                        count_neg[vocab_neg.index(subword)]+=1
                    else:
                        vocab_neg.append(subword)
                        count_neg.append(1)
        if i not in T_vocab:
            T_vocab.append(i)
       

for eg in XTR_neu:
    words= eg.split()
    for i in words:
        if i in stop_words or i.isnumeric() or i in exclude:
            continue
        if d.check(i):
            if i in vocab_neu:
                count_neu[vocab_neu.index(i)]+=1
            else:
                vocab_neu.append(i)
                count_neu.append(1)
        else:
            if (d.suggest(i)):
                suggested = ((d.suggest(i)[0]).lower()).split()
                for subword in suggested:
                    if subword in vocab_neu:
                        count_neu[vocab_neu.index(subword)]+=1
                    else:
                        vocab_neu.append(subword)
                        count_neu.append(1)
        if i not in T_vocab:
            T_vocab.append(i)

In [228]:
#Positive generator with Add-1 Smoothing
def PosGen(test):
    P_pos= len(XTR_pos)/(len(XTR_pos)+len(XTR_neg)+len(XTR_neu))
    words= test.split()
    for i in words:
        if not d.check(i):
            if (d.suggest(i)):
                suggested = ((d.suggest(i)[0]).lower()).split()
                for subword in suggested:
                    if subword in T_vocab:
                        if subword in vocab_pos:
                            P_pos *= ((count_pos[vocab_pos.index(subword)])+1)/sum(count_pos,len(T_vocab))
                        else:
                            P_pos *= (1)/sum(count_pos,len(T_vocab))
                            
        
        else:
            if i in T_vocab:
                if i in vocab_pos:
                    P_pos *= ((count_pos[vocab_pos.index(i)])+1)/sum(count_pos,len(T_vocab))
                else:
                    P_pos *= (1)/sum(count_pos,len(T_vocab))
        
    return P_pos
                    

    

In [229]:
#Negative generator with Add-1 Smoothing
def NegGen(test):
    P_neg= len(XTR_neg)/(len(XTR_pos)+len(XTR_neg)+len(XTR_neu))
    words= test.split()
    for i in words:
        if not d.check(i):
            if (d.suggest(i)):
                suggested = ((d.suggest(i)[0]).lower()).split()
                for subword in suggested:
                    if subword in T_vocab:
                        if subword in vocab_neg:
                            P_neg *= ((count_neg[vocab_neg.index(subword)])+1)/sum(count_neg,len(T_vocab))
                        else:
                            P_neg *= (1)/sum(count_neg,len(T_vocab))
        
        else:
            if i in T_vocab:
                if i in vocab_neg:
                    P_neg *= (count_neg[vocab_neg.index(i)]+1)/sum(count_neg,len(T_vocab))
                else:
                    P_neg *= (1)/sum(count_neg,len(T_vocab))
        
    return P_neg
               

In [238]:
#Neutral generator with Add-1 Smoothing
def NeuGen(test):
    P_neu= len(XTR_neu)/(len(XTR_pos)+len(XTR_neg)+len(XTR_neu))
    words= test.split()
    for i in words:
        if not d.check(i):
            if (d.suggest(i)):
                suggested = ((d.suggest(i)[0]).lower()).split()
                for subword in suggested:
                    if subword in T_vocab:
                        if subword in vocab_neu:
                            P_neu *= ((count_neu[vocab_neu.index(subword)])+1)/sum(count_neu,len(T_vocab))
                        else:
                            P_neu *= (1)/sum(count_neu,len(T_vocab))
        
        else:
            if i in T_vocab:
                if i in vocab_neu:
                    P_neu *= (count_neu[vocab_neu.index(i)]+1)/sum(count_neu,len(T_vocab))
                else:
                    P_neu *= (1)/sum(count_neu,len(T_vocab))
        
    return P_neu
               

In [296]:
#Classifier Function, input is a raw tweet, output is the sentiment

def classify(test):
    X= np.zeros(3)
    X[1] = PosGen(test)
    X[2] = NegGen(test)
    X[0] = NeuGen(test)
    return (np.ndarray.tolist(X).index(max(X)))
    
    

In [297]:
#Macro ACCURACY CALCULATED HERE
p=0
ng=0
nu=0
for i in ((XTE_pos)):
    if (classify(i) == 1):
        p+=1
for i in ((XTE_neg)):
    if(classify(i) == 2):
        ng+=1
for i in ((XTE_neu)):
    if(classify(i) == 0):
        nu+=1

Accuracy = (p+ng+nu)/(len(XTE_neg)+len(XTE_pos)+len(XTE_neu))

In [298]:
#Measures Calculated here 
tp0 = 0
fp0 = 0
tn0 = 0
fn0 = 0

tp1 = 0
fp1 = 0
tn1 = 0
fn1 = 0

tp2 = 0
fp2 = 0
tn2 = 0
fn2 = 0
c=0

for i in (XTE_neu):
    pred = classify(i)
    if (pred == 0):
        c+=1
        tp0+=1 ###
        tn1+=1
        tn2+=1
        ##############
    if (pred != 0):
        fn0+=1
        if pred==1:
            c+=1
            fp1+=1 
            tn2+=1
        if pred==2:
            c+=1
            fp2+=1
            tn1+=1

for i in (XTE_pos):
    pred = classify(i)
    if pred==1:
        c+=1
        tp1+=1 ###
        tn0+=1
        tn2+=1
    if (pred != 1):
        fn1+=1
        if pred==0:
            c+=1
            fp0+=1 
            tn2+=1
        if pred==2:
            c+=1
            fp2+=1
            tn0+=1

for i in (XTE_neg):
    pred = classify(i)
    if pred==2:
        c+=1
        tp2+=1 ###
        tn0+=1
        tn1+=1
    if (pred != 2):
        fn2+=1
        if pred==0:
            c+=1
            fp0+=1 
            tn1+=1
        if pred==1:
            c+=1
            fp1+=1
            tn0+=1

In [299]:
#MACROAVERAGE MEASURES
#Precision 
P0 = tp0/(fp0+tp0)
P1 = tp1/(fp1+tp1)
P2 = tp2/(fp2+tp2)

#Recall
R0 = tp0/(tp0+fn0)
R1 = tp1/(tp1+fn1)
R2 = tp2/(tp2+fn2)

#F-1 Measure
F1_0 = (2*R0*P0)/(R0+P0)
F1_1 = (2*R1*P1)/(R1+P1)
F1_2 = (2*R2*P2)/(R2+P2)

F1 = (F1_0+F1_1+F1_2)/3

print('Macro Average Measures :')
print('Precision = ', math.ceil(((P0+P1+P2)/3)*100),'%')
print('Recall = ', math.ceil(((R0+R1+R2)/3)*100),'%')
print('Accuracy = ', math.ceil(Accuracy*100),'%')
print('F1- Measure = ', math.ceil(F1*100),'%' )

Macro Average Measures :
Precision =  76 %
Recall =  61 %
Accuracy =  77 %
F1- Measure =  65 %


In [300]:
#MicroAverage Measures 
tps=tp0+tp1+tp2
fps=fp0+fp1+fp2
fns=fn0+fn1+fn2

Pr = (tps)/(tps+fps)
Re = (tps)/(tps+fns)

MicF1 = (2*Pr*Re)/(Pr+Re)

print('Micro Average Measures :')
print('Precision = ', math.ceil((Pr)*100),'%')
print('Recall = ', math.ceil((Re)*100),'%')
print('Accuracy = ', math.ceil((Re)*100),'%')
print('F1- Measure = ', math.ceil(MicF1*100),'%' )

Micro Average Measures :
Precision =  77 %
Recall =  77 %
Accuracy =  77 %
F1- Measure =  77 %
